In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point
from geopandas import GeoDataFrame
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [45]:
def obtener_rmse(col_true, col_pred):
    return mean_squared_error(col_true, col_pred)**0.5

In [3]:
propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/datos para el tp2/set_datos_propiedades.csv')

In [4]:
propiedades.drop(['lat','lon','fecha_de_publicacion', 'expenses', 'state_name',\
                  'distancia_villas', 'description'], axis = 1, inplace = True)
propiedades = propiedades.loc[propiedades.superficie.notnull(),:]

## Convierto los indices con preprocesamiento

In [5]:
barrios = {}
for x in list(propiedades.place_name_encoded):
    if x not in barrios:
        barrios[x] = [x]

In [6]:
tipos = preprocessing.OneHotEncoder()
tipos.fit([[0],[1],[2], [3]]) 

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [7]:
pre_barrios = preprocessing.OneHotEncoder()
pre_barrios.fit(barrios.values())

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [8]:
propiedades.loc[:,'place_name_encoded_2'] = propiedades.loc[:,'place_name_encoded']\
.apply(lambda x: pre_barrios.transform([[x]]))

In [9]:
propiedades.loc[:,'property_type_encoded_2'] = propiedades.loc[:,'property_type_encoded']\
.apply(lambda x: tipos.transform([[x]]))

In [10]:
propiedades.to_csv('/home/agustin/Escritorio/datos_indice.csv', index = False)

In [11]:
entrenar = propiedades.loc[(propiedades.Year >= 2016) & ((propiedades.Month < 6) | (propiedades.Year < 2017)),:]

In [12]:
adivinar = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),:]

## Perceptron sin indices de preprocesamiento

In [44]:
def adivinar_precio(fila, perceptron):
    return  perceptron.predict([fila])[0]

In [7]:
condiciones = ['place_name_encoded', 'property_type_encoded', 'superficie', 'gimnasio',\
               'pileta', 'seguridad']

precios = condiciones + ['price_aprox_usd']

In [41]:
funcion = ['adam', 'lbfgs', 'sgd']
activacion = ['relu','identity', 'logistic', 'tanh']
tolerancia = [1e-5, 1e-8, 1e-10]

In [ ]:
impresion = []
for i in range (0, len(funcion)):
    for j in range(0, len(activacion)):
        for k in range(0, len(tolerancia)):
            perceptron = MLPRegressor(solver = funcion[i], activation = activacion[j], tol = tolerancia[k])
            perceptron.fit(entrenar.loc[:,condiciones], entrenar.loc[:, 'price_aprox_usd'])
            adivinar.loc[:,'precio_adivinado_perceptron'] = adivinar.loc[:, condiciones]\
            .apply(lambda x: adivinar_precio(x, perceptron), axis = 1)
            error = obtener_rmse(adivinar.loc[:,'price_aprox_usd'], adivinar.loc[:,'precio_adivinado_perceptron'])
            impresion.append((funcion[i], activacion[j], tolerancia[k], error))

In [21]:
for i in range (0, len(impresion)):
    print("Funcion = {} , activacion = {}, tolerancia = {}, error = {}".format(impresion[i][0], impresion[i][1], impresion[i][2], impresion[i][3]))

Funcion = adam , activacion = relu, tolerancia = 1e-05, error = 2856923.30569
Funcion = adam , activacion = relu, tolerancia = 1e-08, error = 7885404.28854
Funcion = adam , activacion = relu, tolerancia = 1e-10, error = 983892.01563
Funcion = adam , activacion = identity, tolerancia = 1e-05, error = 8068786.87283
Funcion = adam , activacion = identity, tolerancia = 1e-08, error = 6847308.97947
Funcion = adam , activacion = identity, tolerancia = 1e-10, error = 7189725.68045
Funcion = adam , activacion = logistic, tolerancia = 1e-05, error = 446011.124687
Funcion = adam , activacion = logistic, tolerancia = 1e-08, error = 446001.878031
Funcion = adam , activacion = logistic, tolerancia = 1e-10, error = 446003.376714
Funcion = adam , activacion = tanh, tolerancia = 1e-05, error = 445965.857984
Funcion = adam , activacion = tanh, tolerancia = 1e-08, error = 445964.564555
Funcion = adam , activacion = tanh, tolerancia = 1e-10, error = 445965.56429
Funcion = lbfgs , activacion = relu, toler

#### Algunos de sgd no corrieron porque tiraron error. El mejor resultado se obtuvo con solver = lbfgs, activation = logistic y tol = 1e-8

## Corro nuevamente lbfgs pero con iteraciones de aprendizaje

In [46]:
impresion_2 = []
for k in range(0, len(tolerancia)):
        perceptron = MLPRegressor(solver = 'lbfgs', activation = 'logistic', tol = tolerancia[k], warm_start = True)
        for j in range(15):
            perceptron.fit(entrenar.loc[:,condiciones], entrenar.loc[:, 'price_aprox_usd'])
        adivinar.loc[:,'precio_adivinado_perceptron'] = adivinar.loc[:, condiciones]\
        .apply(lambda x: adivinar_precio(x, perceptron), axis = 1)
        error = obtener_rmse(adivinar.loc[:,'price_aprox_usd'], adivinar.loc[:,'precio_adivinado_perceptron'])
        impresion_2.append(('lbfgs', 'logistic', tolerancia[k], error))

In [47]:
for i in range (0, len(impresion_2)):
    print("Funcion = {} , activacion = {}, tolerancia = {}, error = {}".format(impresion_2[i][0], impresion_2[i][1], impresion_2[i][2], impresion_2[i][3]))
    #print("Funcion = {} , activacion = {}, tolerancia = {}, error = {}".format(impresion_2[i]))

Funcion = lbfgs , activacion = logistic, tolerancia = 1e-05, error = 374794.999012
Funcion = lbfgs , activacion = logistic, tolerancia = 1e-08, error = 387524.296117
Funcion = lbfgs , activacion = logistic, tolerancia = 1e-10, error = 383514.002996


#### Despues veo de ejecutar con los indices de preprocesamiento. Pero seguro tarde mucho tiempo mas

## Calculo con los verdaderos datos con indices simples

In [4]:
entrenar = propiedades.loc[(propiedades.Year >= 2016) & ((propiedades.Month >= 6) | (propiedades.Year == 2017)),:]

In [8]:
perceptron = MLPRegressor(solver = 'lbfgs', activation = 'logistic', tol=1e-10,  warm_start = True)
for k in range(6):
    perceptron.fit(entrenar.loc[:,condiciones], entrenar.loc[:, 'price_aprox_usd'])

In [9]:
analizar = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/datos para el tp2/properati_dataset_modificado.csv')

In [10]:
analizar.keys()

Index([u'id', u'created_on', u'property_type', u'place_name',
       u'place_with_parent_names', u'state_name', u'lat', u'lon',
       u'surface_total_in_m2', u'surface_covered_in_m2', u'floor', u'rooms',
       u'expenses', u'description', u'seguridad', u'gimnasio', u'aire',
       u'pileta', u'cochera', u'superficie', u'property_type_encoded',
       u'place_name_encoded'],
      dtype='object')

In [12]:
analizar.loc[:,'price_usd'] = analizar.loc[:,condiciones].apply(lambda x: perceptron.predict([x])[0],axis = 1)

In [14]:
analizar.price_usd.describe()

count     14166.000000
mean     273737.920575
std       13839.190497
min      257696.031603
25%      263776.557855
50%      263816.677580
75%      282017.209480
max      296299.883654
Name: price_usd, dtype: float64

In [16]:
resultado = analizar.loc[:,['id','price_usd']]

In [17]:
resultado.to_csv('Perceptron_resultados.csv', index = False)

## Con preprocesamiento de standarizacion

In [5]:
entrenar = propiedades.loc[((propiedades.Year == 2016) & (propiedades.Month >= 6)) | ((propiedades.Year == 2017) & (propiedades.Month < 6)),:]
adivinar = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month >= 6),:]

In [11]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(entrenar.loc[:,condiciones])  
#X_train = scaler.transform(condiciones)  

StandardScaler(copy=True, with_mean=True, with_std=True)

In [40]:
perceptron = MLPRegressor(solver = 'lbfgs', activation = 'logistic', tol=1e-10,  warm_start = True)
for k in range(6):
    perceptron.fit(scaler.transform(entrenar.loc[:,condiciones]), entrenar.loc[:, 'price_aprox_usd'])
    print(k)

In [34]:
adivinar.loc[:,'precio_adivinado_perceptron'] = adivinar.loc[:, condiciones]\
.apply(lambda x: perceptron.predict(scaler.transform([x]))[0],axis = 1)
obtener_rmse(adivinar.loc[:,'price_aprox_usd'], adivinar.loc[:,'precio_adivinado_perceptron'])

371377.69918790885

In [35]:
perceptron.fit(scaler.transform(adivinar.loc[:,condiciones]), adivinar.loc[:, 'price_aprox_usd'])

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=1e-10, validation_fraction=0.1,
       verbose=False, warm_start=True)

In [36]:
adivinar.loc[:,'precio_adivinado_perceptron'] = adivinar.loc[:, condiciones]\
.apply(lambda x: perceptron.predict(scaler.transform([x]))[0],axis = 1)
obtener_rmse(adivinar.loc[:,'price_aprox_usd'], adivinar.loc[:,'precio_adivinado_perceptron'])

371163.71740396123

In [37]:
adivinar.precio_adivinado_perceptron.describe()

count     84605.000000
mean     275410.730227
std      111632.520567
min       51843.428600
25%      179065.385049
50%      275235.405024
75%      368577.680155
max      484760.297554
Name: precio_adivinado_perceptron, dtype: float64

In [25]:
analizar = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/datos para el tp2/properati_dataset_modificado.csv')

In [38]:
analizar.loc[:,'price_usd'] = analizar.loc[:,condiciones]\
.apply(lambda x: abs(perceptron.predict(scaler.transform([x]))[0]),axis = 1)

In [39]:
analizar.price_usd.describe()

count     14166.000000
mean     241770.056467
std       95024.671605
min       51843.637336
25%      179065.385049
50%      204477.225855
75%      325592.993634
max      478662.318505
Name: price_usd, dtype: float64

In [30]:
resultado = analizar.loc[:,['id','price_usd']]

In [31]:
resultado.to_csv('Perceptron_resultados_con_standard_scaler.csv', index = False)

## Perceptron con los indices de preprocesamiento

In [13]:
def adivinar_precio_indice(fila, perceptron):
    barrio = fila[0]
    tipo = fila[1]
    columnas = fila[2:]
    l =  list(barrio.toarray()[0]) + list(tipo.toarray()[0]) + columnas
    return  perceptron.predict([l])[0]

In [14]:
condiciones = ['place_name_encoded_2', 'property_type_encoded_2', 'superficie', 'gimnasio',\
               'pileta', 'seguridad']

precios = condiciones + ['price_aprox_usd']

In [15]:
funcion = ['adam', 'lbfgs', 'sgd']
activacion = ['relu','identity', 'logistic', 'tanh']
tolerancia = [1e-5, 1e-8, 1e-10]
impresion = []
#max_iter = 500
#warm.....

In [ ]:
X_true = []
Y_true = []
entrenar.reset_index(drop = True, inplace = True)
for i in range(0,len(entrenar)):
    if np.isnan(entrenar.price_aprox_usd[i]):
        continue
    l = list(entrenar.place_name_encoded_2[i].toarray()[0]) + list(entrenar.property_type_encoded_2[i].toarray()[0])
    l.append(entrenar.superficie[i])
    l.append(entrenar.gimnasio[i])
    l.append(entrenar.pileta[i])
    l.append(entrenar.seguridad[i])
    X_true.append(l)
    Y_true.append(entrenar.price_aprox_usd[i])

In [2]:
propiedades.head()

NameError: name 'propiedades' is not defined

In [16]:
for i in range (0, len(funcion)):
    for j in range(0, len(activacion)):
        for k in range(0, len(tolerancia)):
            perceptron = MLPRegressor(solver = funcion[i], activation = activacion[j], tol = tolerancia[k])
            perceptron.fit(entrenar.loc[:,condiciones], entrenar.loc[:, 'price_aprox_usd'])
            adivinar.loc[:,'precio_adivinado_perceptron'] = adivinar.loc[:, condiciones]\
            .apply(lambda x: adivinar_precio(x, perceptron), axis = 1)
            error = obtener_rmse(adivinar.loc[:,'price_aprox_usd'], adivinar.loc[:,'precio_adivinado_perceptron'])
            impresion.append((funcion[i], activacion[j], tolerancia[k], error))

ValueError: setting an array element with a sequence.

In [ ]:
for i in range (0, len(impresion)):
    print("Funcion = {} , activacion = {}, tolerancia = {}, error = {}".format(impresion[i][0], impresion[i][1], impresion[i][2], impresion[i][3]))

In [27]:
perceptron = MLPRegressor()
perceptron.fit(propiedades.loc[:,condiciones], propiedades.loc[:, 'price_aprox_usd'])

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [39]:
adivinar.loc[:,'precio_adivinado_perceptron'] = adivinar.loc[:, condiciones]\
.apply(lambda x: adivinar_precio_indice(x, perceptron), axis = 1)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [63]:
obtener_rmse(propiedades.loc[:,'price_aprox_usd'], propiedades.loc[:,'precio_adivinado_perceptron'])

416926.48116043647